In [ ]:
# скачиваем файл
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl' -O heart.csv

In [58]:
import numpy as np
import pandas as pd

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [59]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1. Обучите решающее дерево из `sklearn` (`sklearn.tree.DecisionTreeClassifier`) без ограничения на максимальную глубину (`max_depth=None`). В качестве сида поставьте `random_state=13`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

   Найдите долю правильных ответов полученного алгоритма на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [61]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
tree = DecisionTreeClassifier(max_depth=None, random_state=13)

In [63]:
from sklearn.metrics import accuracy_score

In [64]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=13)

In [65]:
round(accuracy_score(y_train, tree.predict(X_train)) * 100,2)

100.0

2. Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Заметно ли переобучение?

In [66]:
round(accuracy_score(y_test, tree.predict(X_test)) * 100,2)

67.11

3. Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:


- `max_depth`: [3, 4, 5, 6, 7, 8, 9, 10, None]
- `max_features`: ['auto', 'log2', None]
- `min_samples_leaf`: range(1, 10)
- `min_samples_split`: range(2, 10)
- `criterion`: ['gini', 'entropy']

    В этом вам поможет метод `sklearn.model_selection.GridSearchCV`. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
    
    1) Создайте решающее дерево - не забудьте поставить `random_state=13`.
    
    2) Задайте `param_grid` - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.
    
    3) Вызовите метод `GridSearchCV` - в качестве параметра `estimator` задайте решающее дерево из первого шага, `param_grid` - сетку из второго. Задайте параметр `cv=5`, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр `scoring='accuracy'`, чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью `fit`.
    
    4) Выведите лучшие параметры с помощью атрибута `best_params_`.
    
    Какое значение глубины дерева получилось оптимальным?

In [71]:
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=13)
tree_params = [
    {
        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
        'max_features': ['log2', None, 'sqrt'],
        # 'max_features': ['log2', None],
        'min_samples_leaf': range(1, 10),
        'min_samples_split': range(2, 10),
        'criterion': ['gini', 'entropy']
    }
]
searcher = GridSearchCV(tree, tree_params, cv=5, scoring='accuracy')

In [72]:
searcher.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=13),
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
                          'max_features': ['log2', None, 'sqrt'],
                          'min_samples_leaf': range(1, 10),
                          'min_samples_split': range(2, 10)}],
             scoring='accuracy')

In [73]:
searcher.best_params_

{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'log2',
 'min_samples_leaf': 3,
 'min_samples_split': 9}

4. Какое лучшее усредненное значение доли правильных ответов получилось на кросс-валидации (для оптимальных значений гиперпараметров)? Вам поможет атрибут `best_score_`. Ответ округлите до двух знаков после запятой и дайте в процентах.

In [74]:
round(searcher.best_score_*100,2)

np.float64(83.24)

5. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

In [81]:
round( accuracy_score(y_train, searcher.predict(X_train)) * 100,2)

90.75

6. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Уменьшилось ли переобучение?

In [82]:
round( accuracy_score(y_test, searcher.predict(X_test)) * 100,2)

73.68

7. Решающее дерево позволяет предсказывать не только классы, но и вероятности классов - с помощью метода `predict_proba`. Посмотрите на вероятности классов полученного решающего дерева и посчитайте значение AUC-ROC. Ответ округлите до двух знаков после запятой.

In [83]:
searcher.predict_proba(X_test)

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.33333333, 0.66666667],
       [0.42857143, 0.57142857],
       [0.8       , 0.2       ],
       [0.75      , 0.25      ],
       [0.28571429, 0.71428571],
       [0.8       , 0.2       ],
       [0.        , 1.        ],
       [0.33333333, 0.66666667],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.66666667, 0.33333333],
       [0.66666667, 0.33333333],
       [0.        , 1.        ],
       [0.57142857, 0.42857143],
       [0.85714286, 0.14285714],
       [0.66666667, 0.33333333],
       [0.66666667, 0.33333333],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.75      , 0.25      ],
       [0.

In [43]:
from sklearn.metrics import roc_auc_score

In [84]:
round(roc_auc_score(y_test, searcher.predict_proba(X_test)[:, 1]),2)

np.float64(0.77)

8. Какой признак является самым важным по мнению полученного решающего дерева? Чтобы это проверить, вам поможет атрибут `feature_importances_`.

In [85]:
max_idx = np.argmax(tree.feature_importances_)
tree.feature_names_in_[max_idx]

'thal'